# 014 – Arduino Test (Pump & Trafo)

Quick smoke-test for both Arduinos:
- **PumpLocker** on `COM19` (temp, fan, flow1, flow2)
- **TrafoLocker** on `COM33` (temp, fan)

In [ ]:
import sys, logging
from pathlib import Path

# Make src importable
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))

from devices.arduino.pump_arduino import PumpArduino
from devices.arduino.trafo_arduino import TrafoArduino

In [ ]:
# Shared logger for both Arduinos
logger = logging.getLogger("arduino_test")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s  %(levelname)-8s  %(message)s"))
    logger.addHandler(handler)

logger.info("Logger ready")

In [ ]:
# Instantiate both Arduinos
pump  = PumpArduino("pump_locker",  port="COM19", logger=logger)
trafo = TrafoArduino("trafo_locker", port="COM33", logger=logger)

print(pump.get_status())
print(trafo.get_status())

In [ ]:
# Connect
pump.connect()
trafo.connect()

In [ ]:
# Read a few lines from each Arduino
import time

for i in range(10):
    p = pump.read_arduino_data()
    t = trafo.read_arduino_data()
    if p:
        print(f"Pump:  {p}")
    if t:
        print(f"Trafo: {t}")
    time.sleep(0.5)

In [ ]:
# Disconnect
pump.disconnect()
trafo.disconnect()